In [1]:
%load_ext autoreload
%autoreload 2

In [25]:
from copy import deepcopy
from io import BytesIO
import json
import os
os.environ['OMP_NUM_THREADS'] = '38'

import igl
import k3d
import numpy as np
import matplotlib.pyplot as plt
import pymesh
from tqdm import tqdm
import torch
import trimesh.transformations as tt
import trimesh
import yaml

In [3]:
def display_sharpness(mesh=None, plot_meshvert=True,
                      samples=None, samples_distances=None,
                      sharp_vert=None, sharp_curves=None,
                      directions=None, directions_width=0.0025,
                      samples_color=0x0000ff, samples_psize=0.002, 
                      mesh_color=0xbbbbbb, meshvert_color=0x666666, meshvert_psize=0.0025,
                      sharpvert_color=0xff0000, sharpvert_psize=0.0025,
                      sharpcurve_color=None, sharpcurve_width=0.0025,
                      as_image=False, plot_height=768,
                      cmap=k3d.colormaps.matplotlib_color_maps.coolwarm_r):
    
    plot = k3d.plot(height=plot_height)
    
    if None is not mesh:
        k3d_mesh = k3d.mesh(mesh.vertices, mesh.faces, color=mesh_color, flat_shading=False)
        plot += k3d_mesh

        if plot_meshvert:
            k3d_points = k3d.points(mesh.vertices, 
                                    point_size=meshvert_psize, color=meshvert_color)
            plot += k3d_points
            k3d_points.shader='flat'

    if None is not samples:
        colors = None
        if None is not samples_distances:
            max_dist = 1.0

            colors = k3d.helpers.map_colors(
                samples_distances, cmap, [0, max_dist]
            ).astype(np.uint32)
            k3d_points = k3d.points(samples, point_size=samples_psize, colors=colors)
        else:
            k3d_points = k3d.points(samples, point_size=samples_psize, color=samples_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not directions:
            vectors = k3d.vectors(
                samples,
                directions * samples_distances[..., np.newaxis],
                use_head=False, 
                line_width=directions_width)
            print(vectors)
            plot += vectors

#             directions_to_plot = np.hstack((samples, samples + directions))
            
#             for i, dir_to_plot in enumerate(directions_to_plot):
#                 dir_to_plot = dir_to_plot.reshape((2, 3))
#                 if np.all(dir_to_plot[0] == dir_to_plot[1]):
#                     continue
#                 color = int(colors[i]) if None is not colors else samples_color
#                 plt_line = k3d.line(dir_to_plot, 
#                                     shader='mesh', width=directions_width, color=color)
#                 plot += plt_line

    if None is not sharp_vert:
        k3d_points = k3d.points(sharp_vert,
                                point_size=sharpvert_psize, color=sharpvert_color)
        plot += k3d_points
        k3d_points.shader='flat'
        
        if None is not sharp_curves:            
            if None is not sharpcurve_color:
                color = sharpcurve_color
            else:
                import randomcolor
                rand_color = randomcolor.RandomColor()
            for i, vert_ind in enumerate(sharp_curves):
                sharp_points_curve = mesh.vertices[vert_ind]
                
                if None is sharpcurve_color:
                    color = rand_color.generate(hue='red')[0]
                    color = int('0x' + color[1:], 16)
                plt_line = k3d.line(sharp_points_curve, 
                                    shader='mesh', width=sharpcurve_width, color=color)
                plot += plt_line
        
    plot.grid_visible = False
    plot.display()
    
    if as_image:
        plot.fetch_screenshot()
        return Image(data=b64decode(plot.screenshot))

In [4]:
# mm/pixel
HIGH_RES = 0.02
MED_RES = 0.05
LOW_RES = 0.125
XLOW_RES = 0.25

## 1. Working with point patches

In [21]:
from sharpf.data import DataGenerationException
from sharpf.utils.abc_utils.abc.abc_data import ABCModality, ABCChunk, ABC_7Z_FILEMASK
from sharpf.data.annotation import ANNOTATOR_BY_TYPE
from sharpf.data.camera_pose_manager import POSE_MANAGER_BY_TYPE
from sharpf.data.datasets.sharpf_io import save_depth_maps
from sharpf.data.imaging import IMAGING_BY_TYPE
from sharpf.data.noisers import NOISE_BY_TYPE
from sharpf.utils.abc_utils.abc import feature_utils
from sharpf.utils.py_utils.console import eprint_t
from sharpf.utils.py_utils.os import add_suffix
from sharpf.utils.py_utils.config import load_func_from_config
from sharpf.utils.abc_utils.mesh.io import trimesh_load
from sharpf.utils.plotting import display_depth_sharpness, illustrate_camera
from sharpf.utils.camera_utils.camera_pose import camera_to_display
from sharpf.utils.abc_utils.mesh.indexing import reindex_zerobased, compute_relative_indexes
import sharpf.data.data_smells as smells

from sharpf.utils.camera_utils.spherical_spiral_sampling import spherical_spiral_sampling
from sharpf.utils.camera_utils.camera_pose import CameraPose

In [6]:
!ls /data

3D_normals_dataset_generation.ipynb  realworld_sharpf_scans_raw
_SKOLTECH.rar			     scannet
abc				     sharp_features_data
colmap_test			     shm_stbasil
mesh_denoising			     toy
realworld_sharpf_scans


In [7]:
with ABCChunk(['/data/abc/abc_0051_obj_v00.7z', '/data/abc/abc_0051_feat_v00.7z']) as data_holder:
    item = data_holder.get('00510073_951e25d2ded40f22b598f84e_000')
item.item_id

'00510073_951e25d2ded40f22b598f84e_000'

In [8]:
mesh, _, _ = trimesh_load(item.obj)
features = yaml.load(item.feat, Loader=yaml.Loader)

In [9]:
pymesh.detect_self_intersection(mesh)

array([], shape=(0, 2), dtype=int32)

In [10]:
config = {
  "shape_fabrication_extent": 10.0,
  "short_curve_quantile": 0.25,
  "base_n_points_per_short_curve": 8,
  "base_resolution_3d": 0.125,
  "camera_pose": {
    "type": "sphere_spiral_to_origin",
    "n_images": 0,
    "layer_radius": 0.1,
    "resolution": 0.02,
    "n_initial_samples": 8,
    "min_arc_length": 0
#     "type": "composite",
#     "sequences": [
#         {
#             "type": "sphere_spiral_to_origin",
#             "n_images": 0,
#             "layer_radius": 0.1,
#             "resolution": 0.01,
#             "n_initial_samples": 10001,
#             "min_arc_length": 0.1
#         }
#       {
#         "type": "sphere_to_origin",
#         "n_images": 2
#       },
#       {
#         "type": "xy_translation",
#         "n_images": 8*8
#       },
#       {
#         "type": "z_rotation",
#         "n_images": 5
#       }
#     ]
  },
  "imaging": {
    "type": "raycasting",
    "projection": "ortho",
    "resolution_image": 512,
    "resolution_3d": 0.02,
    "fov": [115, 85, 80],
    "validate_image": True
  },
  "noise": {
    "type": "z_direction",
    "scale": 0.0
  },
  "annotation": {
    "type": "surface_based_aabb",
    "distance_upper_bound": 1.0,
    "distance_computation_method": "igl",
  },
  "smell_coarse_surfaces_by_num_edges": {
    "num_edges_threshold": 8
  },
  "smell_coarse_surfaces_by_angles": {
    "max_angle_threshold_degrees": 10.0
  },
  "smell_deviating_resolution": {
    "resolution_3d": 0.02,
    "resolution_deviation_tolerance": 0.01
  },
  "smell_sharpness_discontinuities": { },
  "smell_bad_face_sampling": {
    "min_points_per_face": 0.02,
    "max_points_per_face": 20.0
  },
  "smell_raycasting_background": { },
  "smell_mesh_self_intersections": { },
  "smell_depth_discontinuity": {
    "depth_discontinuity_threshold": 0.5
  }
}

In [13]:
shape_fabrication_extent = config.get('shape_fabrication_extent', 10.0)
base_n_points_per_short_curve = config.get('base_n_points_per_short_curve', 8)
base_resolution_3d = config.get('base_resolution_3d', LOW_RES)

short_curve_quantile = config.get('short_curve_quantile', 0.05)

pose_manager = load_func_from_config(POSE_MANAGER_BY_TYPE, config['camera_pose'])
imaging = load_func_from_config(IMAGING_BY_TYPE, config['imaging'])
noiser = load_func_from_config(NOISE_BY_TYPE, config['noise'])
annotator = load_func_from_config(ANNOTATOR_BY_TYPE, config['annotation'])

{'composite': <class 'sharpf.data.camera_pose_manager.CompositePoseManager'>, 'sphere_to_origin': <class 'sharpf.data.camera_pose_manager.SphereOrientedToWorldOrigin'>, 'sphere_spiral_to_origin': <class 'sharpf.data.camera_pose_manager.SphericalSpiralOrientedToWorldOrigin'>, 'z_rotation': <class 'sharpf.data.camera_pose_manager.ZRotationInCameraFrame'>, 'xy_translation': <class 'sharpf.data.camera_pose_manager.XYTranslationInCameraFrame'>} {'type': 'sphere_spiral_to_origin', 'n_images': 0, 'layer_radius': 0.1, 'resolution': 0.02, 'n_initial_samples': 8, 'min_arc_length': 0}
{'raycasting': <class 'sharpf.data.imaging.RaycastingImaging'>} {'type': 'raycasting', 'projection': 'ortho', 'resolution_image': 512, 'resolution_3d': 0.02, 'fov': [115, 85, 80], 'validate_image': True}
{'no_noise': <class 'sharpf.data.noisers.NoNoise'>, 'isotropic_gaussian': <class 'sharpf.data.noisers.IsotropicGaussianNoise'>, 'normals_gaussian': <class 'sharpf.data.noisers.NormalsGaussianNoise'>, 'z_direction': 

In [14]:
smell_coarse_surfaces_by_num_edges = smells.SmellCoarseSurfacesByNumEdges.from_config(config['smell_coarse_surfaces_by_num_edges'])
smell_coarse_surfaces_by_angles = smells.SmellCoarseSurfacesByAngles.from_config(config['smell_coarse_surfaces_by_angles'])
smell_deviating_resolution = smells.SmellDeviatingResolution.from_config(config['smell_deviating_resolution'])
smell_sharpness_discontinuities = smells.SmellSharpnessDiscontinuities.from_config(config['smell_sharpness_discontinuities'])
smell_bad_face_sampling = smells.SmellBadFaceSampling.from_config(config['smell_bad_face_sampling'])
smell_raycasting_background = smells.SmellRaycastingBackground.from_config(config['smell_raycasting_background'])
smell_depth_discontinuity = smells.SmellDepthDiscontinuity.from_config(config['smell_depth_discontinuity'])
smell_mesh_self_intersections = smells.SmellMeshSelfIntersections.from_config(config['smell_mesh_self_intersections'])


In [15]:
def scale_mesh(mesh, features, shape_fabrication_extent, resolution_3d,
               short_curve_quantile=0.05, n_points_per_short_curve=4):
    # compute standard size spatial extent
    mesh_extent = np.max(mesh.bounding_box.extents)
    mesh = mesh.apply_scale(shape_fabrication_extent / mesh_extent)

    # compute lengths of curves
    sharp_curves_lengths = feature_utils.get_curves_extents(mesh, features)

    least_len = np.quantile(sharp_curves_lengths, short_curve_quantile)
    least_len_mm = resolution_3d * n_points_per_short_curve

    scale = least_len_mm / least_len
    mesh = mesh.apply_scale(scale)

    return mesh, scale


In [16]:
mesh, mesh_scale = scale_mesh(mesh, features, shape_fabrication_extent, base_resolution_3d,
                              short_curve_quantile=short_curve_quantile,
                              n_points_per_short_curve=base_n_points_per_short_curve)

mesh = mesh.apply_translation(-mesh.vertices.mean(axis=0))

In [17]:
processed_mesh = trimesh.base.Trimesh(vertices=mesh.vertices, faces=mesh.faces, process=True, validate=True)
if processed_mesh.vertices.shape != mesh.vertices.shape or \
        processed_mesh.faces.shape != mesh.faces.shape or not mesh.is_watertight:
    raise DataGenerationException('Will not process mesh {}: likely the mesh is broken'.format(data['item_id']))

has_smell_mismatching_surface_annotation = any([
    np.array(np.unique(mesh.faces[surface['face_indices']]) != np.sort(surface['vert_indices'])).all()
    for surface in features['surfaces']
])
has_smell_mesh_self_intersections = smell_mesh_self_intersections.run(mesh)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  


In [18]:
display_sharpness(mesh)

/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float32". A coerced copy has been created.
  np.dtype(self.dtype).name))
/home/user/miniconda/envs/py36/lib/python3.6/site-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "int64" does not match required type "uint32". A coerced copy has been created.
  np.dtype(self.dtype).name))


Output()

In [19]:

non_annotated_patches = []
# generate camera poses
pose_manager.prepare(mesh)
for pose_idx, camera_pose in enumerate(pose_manager):
    eprint_t("Computing images from pose {pose_idx}".format(pose_idx=pose_idx))

    # extract neighbourhood
    try:
        image, points, normals, mesh_face_indexes = \
            imaging.get_image_from_pose(mesh, camera_pose, return_hit_face_indexes=True)
    except DataGenerationException as e:
        eprint_t(str(e))
        continue

    nbhood, mesh_vertex_indexes, mesh_face_indexes = \
        feature_utils.submesh_from_hit_surfaces(mesh, features, mesh_face_indexes)

    has_smell_coarse_surfaces_by_num_edges = smell_coarse_surfaces_by_num_edges.run(mesh, mesh_face_indexes, features)
    has_smell_coarse_surfaces_by_angles = smell_coarse_surfaces_by_angles.run(mesh, mesh_face_indexes, features)
    has_smell_deviating_resolution = smell_deviating_resolution.run(points)
    has_smell_bad_face_sampling = smell_bad_face_sampling.run(nbhood, points)
    has_smell_raycasting_background = smell_raycasting_background.run(image)
    has_smell_depth_discontinuity = smell_depth_discontinuity.run(image)

    # create annotations: condition the features onto the nbhood
    nbhood_features = feature_utils.compute_features_nbhood(
        mesh, features, mesh_face_indexes, mesh_vertex_indexes=mesh_vertex_indexes)

    # remove vertices lying on the boundary (sharp edges found in 1 face only)
    nbhood_features = feature_utils.remove_boundary_features(nbhood, nbhood_features, how='edges')

    # create a noisy sample
    noisy_points = noiser.make_noise(
        camera_pose.world_to_camera(points),
        normals,
        z_direction=np.array([0., 0., -1.]))

    # convert everything to images
    ray_indexes = np.where(image.ravel() != 0)[0]
    noisy_image = imaging.points_to_image(noisy_points, ray_indexes)
    normals_image = imaging.points_to_image(normals, ray_indexes, assign_channels=[0, 1, 2])

    # compute statistics
    num_sharp_curves = len([curve for curve in nbhood_features['curves'] if curve['sharp']])
    num_surfaces = len(nbhood_features['surfaces'])

    patch_info = {
        'image': noisy_image,
        'normals': normals_image,
        # 'distances': distances_image,
        # 'directions': directions_image,
        # 'item_id': item.item_id,
        'ray_indexes': ray_indexes,
        'orig_vert_indices': mesh_vertex_indexes,
        'orig_face_indexes': mesh_face_indexes,
        # 'has_sharp': has_sharp,
        'num_sharp_curves': num_sharp_curves,
        'num_surfaces': num_surfaces,
        'camera_pose': camera_pose.camera_to_world_4x4,
        'mesh_scale': mesh_scale,
        'has_smell_coarse_surfaces_by_num_faces': has_smell_coarse_surfaces_by_num_edges,
        'has_smell_coarse_surfaces_by_angles': has_smell_coarse_surfaces_by_angles,
        'has_smell_deviating_resolution': has_smell_deviating_resolution,
        # 'has_smell_sharpness_discontinuities': has_smell_sharpness_discontinuities,
        'has_smell_bad_face_sampling': has_smell_bad_face_sampling,
        'has_smell_mismatching_surface_annotation': has_smell_mismatching_surface_annotation,
        'has_smell_raycasting_background': has_smell_raycasting_background,
        'has_smell_depth_discontinuity': has_smell_depth_discontinuity,
        'has_smell_mesh_self_intersections': has_smell_mesh_self_intersections,
        'nbhood': nbhood,
        'nbhood_features': nbhood_features,
    }
    non_annotated_patches.append(patch_info)


13.09.2021 10:56:41.974119  MainProcess Computing images from pose 0
13.09.2021 10:56:43.327959  MainProcess Computing images from pose 1
13.09.2021 10:56:44.328332  MainProcess Computing images from pose 2
13.09.2021 10:56:45.464664  MainProcess Computing images from pose 3
13.09.2021 10:56:46.357270  MainProcess Computing images from pose 4
13.09.2021 10:56:47.364362  MainProcess Computing images from pose 5
13.09.2021 10:56:48.565500  MainProcess Computing images from pose 6


In [22]:

whole_model_points, whole_model_point_indexes = [], []
n_points = 0
for patch in non_annotated_patches:
    image = patch['image']
    camera_to_world_4x4 = patch['camera_pose']
    points_in_camera_frame = imaging.image_to_points(image)
    camera_pose = CameraPose(camera_to_world_4x4)
    points_in_world_frame = camera_pose.camera_to_world(points_in_camera_frame)
    whole_model_points.append(points_in_world_frame)
    whole_model_point_indexes.append(
        np.arange(n_points, n_points + len(points_in_world_frame)))
    n_points += len(points_in_world_frame)
whole_model_points = np.concatenate(whole_model_points)


In [23]:
display_sharpness(mesh, samples=whole_model_points)

Output()

In [26]:
%%time 

patch = non_annotated_patches[0]

nbhood = patch['nbhood']
nbhood_features = patch['nbhood_features']

distance_sq, face_indexes, _ = igl.point_mesh_squared_distance(
    whole_model_points,
    nbhood.vertices,
    nbhood.faces)

CPU times: user 1.74 s, sys: 36 ms, total: 1.78 s
Wall time: 156 ms


In [27]:
%%time 

indexes = np.where(np.sqrt(distance_sq) < HIGH_RES / 100)[0]
noisy_points, normals = whole_model_points[indexes], nbhood.face_normals[face_indexes[indexes]]


CPU times: user 72 ms, sys: 292 ms, total: 364 ms
Wall time: 23 ms


In [28]:
annotator.distance_computation_method

'igl'

In [29]:
%%time

try:
    distances, directions, has_sharp = annotator.annotate(nbhood, nbhood_features, noisy_points)
except DataGenerationException as e:
    eprint_t(str(e))


ValueError: distance_computation_method unknown

In [31]:
!git log

commit 670cd5b36dfa2f25b19f4d91282a292c027b09a3
Merge: 0479a4a 08b30f3
Author: emil <emilzq@bk.ru>
Date:   Thu Jul 22 10:32:42 2021 +0000

    Merge branch 'feature/data-quality-naive' of https://github.com/artonson/sharp_features into feature/data-quality-naive

commit 0479a4abb22c67849939f17126fd8e6d18a27380
Author: emil <emilzq@bk.ru>
Date:   Thu Jul 22 10:30:32 2021 +0000

    add collect metrics

commit 08b30f3582f57a335a267fcd8767087a4c232c1f
Author: e.bogomolov <e.bogomolov@an01.zhores>
Date:   Wed Jul 21 13:37:09 2021 +0300

    add resolution multiplier flag

commit 261aa380c256c8be574408967457b7df52ab068b
Author: e.bogomolov <e.bogomolov@an01.zhores>
Date:   Wed Jul 21 12:50:13 2021 +0300

    fix generator

commit cc328787b18f2763e0c85adb10cb1af16d0ca596
Author: e.bogomolov <e.bogomolov@an01.zhores>
Date:   Tue Jul 20 19:43:30 2021 +0300

    fix args

commit e212b35403fc01ff3713232b85367dae726d1e98
Author: emil <emilzq@bk.ru>
Date:   Tue Jul 20 16:39:31 2021 +0000

    fix 


commit 97d8825c48a56cd72d5e79bbdc1fd0bddfe40eaf
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Wed May 19 20:01:37 2021 +0300

    add -tk and -pk parameters to compute_metrics_ruslan.py

commit f0b11aaa7cafbe62a9c2fac80d419d39cfe6efc4
Author: emil <emilzq@bk.ru>
Date:   Wed May 19 13:50:01 2021 +0000

    add binary predictions flag

commit 90c65014fced297439bab08bbb1789e08adb031b
Author: emil <emilzq@bk.ru>
Date:   Wed May 19 11:53:46 2021 +0000

    modify pienet script

commit 27985e6f1e4c23340e742408215e4f8ffc0f28ab
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Wed May 19 02:12:44 2021 +0300

    stuff for computing on cluster

commit c3bd8b712cd8f8d32569a122dc8f5e87263e3761
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Wed May 19 00:30:34 2021 +0300

    update compute_metrics.sbatch.sh

commit 4852eb4eb472737e301fc74aa8a5966848aa4971
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Wed May 19 00:20:28 2021 +0300

    true to make compute_metrics.


commit 2f6f1ae19b08de29a35b4681599649c23c7260da
Merge: bc58115 1c6e01d
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Wed May 12 12:59:49 2021 +0300

    Merge remote-tracking branch 'origin/feature/data-quality-naive' into feature/data-quality-naive

commit bc58115fc1fbf463282a1aa2dd079dc0052bf16a
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Wed May 12 12:59:35 2021 +0300

    export_mesh_for_rendering.py

commit 1c6e01d56a38c0bd451736712cbfdeac3a487659
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Wed May 12 12:43:54 2021 +0300

    fix generate_corners

commit 172d62f377ad7348e187b325094a85301a12e833
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Wed May 12 11:05:56 2021 +0300

    dataset_config

commit 9d2b9c9e0b900ac36d7422df93f65964f2f52fc5
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Wed May 12 11:05:16 2021 +0300

    data_dir

commit 24ada4f3fb12aa035736cf2604598ea3f5668f8e
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   We


commit 7a37f860a2dd9169582d5b3dbb8e43ccb9b1b4f7
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Fri Apr 23 19:01:00 2021 +0300

    add display_patch_decomposition

commit 93d279454f3ce7931a325e1c5f91e0be156dedcf
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Fri Apr 23 18:59:39 2021 +0300

    fix distance_computation_method

commit 401c33bb499e4b299e14af5aaeb6a5186c90c08a
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Fri Apr 23 18:53:18 2021 +0300

    fix import

commit fd67894d0f8908736be2bba99c095ebec1b89b91
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Fri Apr 23 18:52:00 2021 +0300

    specify distance_computation_method

commit bc766da9b7018458f1dc89f6388c73ce3d1d9cc9
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Fri Apr 23 18:47:32 2021 +0300

    add distance scale ratio to fuse images

commit 7e38271d487ffa3108b0573d474ae063124cd5fe
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Fri Apr 23 18:46:07 2021 +0300

    update an

Merge: ea038dc fbc6ab2
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Sat Oct 24 20:45:58 2020 +0300

    Merge branch 'feature/whole_model_annotation' into feature/data-quality-naive

commit ea038dcd0a3374eec58afc5dc1b7b131dc180421
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Fri Oct 23 18:24:33 2020 +0300

     * update generate_pointcloud_data.py to try-catch has_smell_sharpness_discontinuities

commit 545c6ddd8cf980448d3a8d132783efa6d9b13622
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Fri Oct 23 18:23:00 2020 +0300

    update all data generation scripts

commit ddf2dac5cf393e3a416008fccd0add4065666884
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Fri Oct 23 18:19:32 2020 +0300

    check for need decode in trimesh_load

commit 028494221e4cfa46d13602aff7355597438776af
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Fri Oct 23 18:17:02 2020 +0300

    update data generation making it robust to bugs in sharpness discontinuities computation

Merge: ef1b370 d0d7fd0
Author: Alexey Artemov <a.artemov@skoltech.ru>
Date:   Thu Apr 9 12:26:58 2020 +0300

    Merge branch 'feature/ec_net_wrapper'

commit 0a71ee187f04001b94fa7ce418774cc64f5178ad
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Wed Apr 8 18:15:28 2020 +0300

    update metrics computation and report generation script

commit 9786b28e04b442f724e781d8b6a57ed7daedab07
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Mon Apr 6 16:03:47 2020 +0300

    fix passing data and target thru torch tensor

commit ab9887013ab034876f2e04a6fd3e94bef1ce1a20
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Mon Apr 6 02:03:22 2020 +0300

    fix verbose

commit 01240ee5335236a4f7c5de7e80b03d4c387d4bac
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Mon Apr 6 02:01:59 2020 +0300

    bugfix

commit dfa35ac985890deecb64b7a86d033e98900b9425
Author: Alexey Artemov <A.Artemov@skoltech.ru>
Date:   Mon Apr 6 02:01:32 2020 +0300

    bugfix

commit c4b6acebed3dd38e65

In [ ]:

# has_smell_sharpness_discontinuities = smell_sharpness_discontinuities.run(noisy_points, distances)

# patch = {
#     'distances': np.array(distances).astype(np.float64),
#     'directions': np.array(directions).astype(np.float64),
#     'has_sharp': has_sharp,
#     'has_smell_sharpness_discontinuities': has_smell_sharpness_discontinuities,
#     'indexes': indexes
# }




In [ ]:

def pointset_edgeset_distances_projections(points, edges_mesh):
    """Compute the distances and projections using libigl's
    functionality available in the method `point_mesh_squared_distance`:
    triangle [1 2 2] is treated as a segment [1 2]
    (see docs at https://libigl.github.io/libigl-python-bindings/igl_docs/#point_mesh_squared_distance )."""

    distances, _, projections = igl.point_mesh_squared_distance(
        points,
        edges_mesh.vertices,
        edges_mesh.faces)
    return distances, projections


In [ ]:


def get_sharp_edge_endpoints_degen(mesh, features):
    """For computing distances using
    https://libigl.github.io/libigl-python-bindings/igl_docs/#point_mesh_squared_distance."""
    sharp_edge_indexes = np.concatenate([
        mesh.edges_unique[
            np.where(
                np.all(np.isin(mesh.edges_unique, curve['vert_indices']), axis=1)
            )[0]
        ]
        for curve in features['curves'] if curve['sharp']])

    sharp_vert_indexes = np.unique(sharp_edge_indexes)
    sharp_edge_indexes = reindex_array(sharp_edge_indexes, sharp_vert_indexes)
    sharp_edge_vertices = mesh.vertices[sharp_vert_indexes]
    sharp_face_indexes = np.hstack((sharp_edge_indexes, np.atleast_2d(sharp_edge_indexes[:, 1]).T))

    edge_pseudo_mesh = trimesh.base.Trimesh(
        vertices=sharp_edge_vertices,
        faces=sharp_face_indexes,
        process=False,
        validate=False)
    return edge_pseudo_mesh


In [ ]:
import igl

In [ ]:
%%time

distances, _, projections = igl.point_mesh_squared_distance(
    whole_model_points[indexes],
    sharp_edge_vertices, 
    sharp_face_indexes
)

In [ ]:
display_sharpness(
    nbhood, plot_meshvert=False,
    samples=whole_model_points[indexes],
    samples_distances=distances,
    samples_psize=0.02,
    sharp_vert=projections,
    sharpvert_psize=0.02)

In [ ]:
display_sharpness(
    nbhood, 
    samples=whole_model_points[indexes],
    samples_distances=distances,
    samples_psize=0.02,
    sharp_vert=projections,
    sharpvert_psize=0.02)